In [1]:
%%file keysched.m
function w = keysched(key)
    global S_box;
    global m;
    global polyaes;

    subkey = uint32(key);
    rcon = gf(1, m, polyaes);

    for r = 1:10
        % Rotword
        subword = subkey([2 3 4 1], end);
        % Subword
        subword = S_box(subword + 1);
        % Rcon
        subword(1) = bitxor(subword(1), rcon.x);
        w(:, 1, r) = bitxor(subkey(:, 1), subword);
        subword = w(:, 1, r);

        for wd = 2:4
            w(:, wd, r) = bitxor(subkey(:, wd), subword);
            subword = w(:, wd, r);
        end

        subkey = w(:, :, r);
        rcon = gf(2, m, polyaes) * rcon;
    end

end


Created file 'C:\Users\Mathis\Desktop\Projet EMA\keysched.m'.


In [2]:
%%file keysched2.m
function w = keysched2(key)

    S_box = uint32([99, 124, 119, 123, 242, 107, 111, 197, 48, 1, 103, 43, 254, 215, 171, 118, 202, 130, 201, 125, 250, 89, 71, 240, 173, 212, 162, 175, 156, 164, 114, 192, 183, 253, 147, 38, 54, 63, 247, 204, 52, 165, 229, 241, 113, 216, 49, 21, 4, 199, 35, 195, 24, 150, 5, 154, 7, 18, 128, 226, 235, 39, 178, 117, 9, 131, 44, 26, 27, 110, 90, 160, 82, 59, 214, 179, 41, 227, 47, 132, 83, 209, 0, 237, 32, 252, 177, 91, 106, 203, 190, 57, 74, 76, 88, 207, 208, 239, 170, 251, 67, 77, 51, 133, 69, 249, 2, 127, 80, 60, 159, 168, 81, 163, 64, 143, 146, 157, 56, 245, 188, 182, 218, 33, 16, 255, 243, 210, 205, 12, 19, 236, 95, 151, 68, 23, 196, 167, 126, 61, 100, 93, 25, 115, 96, 129, 79, 220, 34, 42, 144, 136, 70, 238, 184, 20, 222, 94, 11, 219, 224, 50, 58, 10, 73, 6, 36, 92, 194, 211, 172, 98, 145, 149, 228, 121, 231, 200, 55, 109, 141, 213, 78, 169, 108, 86, 244, 234, 101, 122, 174, 8, 186, 120, 37, 46, 28, 166, 180, 198, 232, 221, 116, 31, 75, 189, 139, 138, 112, 62, 181, 102, 72, 3, 246, 14, 97, 53, 87, 185, 134, 193, 29, 158, 225, 248, 152, 17, 105, 217, 142, 148, 155, 30, 135, 233, 206, 85, 40, 223, 140, 161, 137, 13, 191, 230, 66, 104, 65, 153, 45, 15, 176, 84, 187, 22])';
    %Rcon=uint32([1,2,4,8,16,32,64,128,27,54]);
    Rcon = uint32([1, 2, 4, 8, 16, 32, 64, 128, 27, 54; 0, 0, 0, 0, 0, 0, 0, 0, 0, 0; 0, 0, 0, 0, 0, 0, 0, 0, 0, 0; 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]);

    k_1 = key(:, 1);
    k_2 = key(:, 2);
    k_3 = key(:, 3);
    k_4 = key(:, 4);

    w(:, :, 1) = key;

    for r = 2:11
        % Operation non linéaire
        k_1 = bitxor(k_1, bitxor(S_box(k_4([2 3 4 1]) + 1), Rcon(:, r - 1)));
        k_2 = bitxor(k_2, k_1);
        k_3 = bitxor(k_3, k_2);
        k_4 = bitxor(k_4, k_3);
        w(:, :, r) = [k_1, k_2, k_3, k_4];
    end

end


Created file 'C:\Users\Mathis\Desktop\Projet EMA\keysched2.m'.


In [1]:
%%file gen_s_box.m

function s_box = gen_s_box
    global m;
    global polyaes;
    Ta = gf([[1 0 0 0 1 1 1 1];
              [1 1 0 0 0 1 1 1];
              [1 1 1 0 0 0 1 1];
              [1 1 1 1 0 0 0 1];
              [1 1 1 1 1 0 0 0];
              [0 1 1 1 1 1 0 0];
              [0 0 1 1 1 1 1 0];
              [0 0 0 1 1 1 1 1]], 1);
    % Calcul de l'inverse
    %s_box = gf((1:2^m-1),m,polyaes).^(-1);
    s_box = gf(ones(1, 2 ^ m - 1), m, polyaes) ./ gf((1:2 ^ m - 1), m, polyaes);
    s_box = [gf(0, m, polyaes) s_box];
    % Calcul de la fonction affine
    s_box = Ta * gf(de2bi(s_box.x', 'right-msb')', 1) + gf([1; 1; 0; 0; 0; 1; 1; 0], 1) * ones(1, 2 ^ m);
    s_box = bi2de(s_box.x', 'right-msb');
end


Created file 'C:\Users\Mathis\Desktop\Projet EMA\gen_s_box.m'.


# Fonctions aide fournies par le professeur
Définition de :
- Shiftrow
- Sbox / InvSbox
- Poids de Hamming

In [ ]:
shiftrow = [1, 6, 11, 16, 5, 10, 15, 4, 9, 14, 3, 8, 13, 2, 7, 12];
SBox = [99, 124, 119, 123, 242, 107, 111, 197, 48, 1, 103, 43, 254, 215, 171, 118, 202, 130, 201, 125, 250, 89, 71, 240, 173, 212, 162, 175, 156, 164, 114, 192, 183, 253, 147, 38, 54, 63, 247, 204, 52, 165, 229, 241, 113, 216, 49, 21, 4, 199, 35, 195, 24, 150, 5, 154, 7, 18, 128, 226, 235, 39, 178, 117, 9, 131, 44, 26, 27, 110, 90, 160, 82, 59, 214, 179, 41, 227, 47, 132, 83, 209, 0, 237, 32, 252, 177, 91, 106, 203, 190, 57, 74, 76, 88, 207, 208, 239, 170, 251, 67, 77, 51, 133, 69, 249, 2, 127, 80, 60, 159, 168, 81, 163, 64, 143, 146, 157, 56, 245, 188, 182, 218, 33, 16, 255, 243, 210, 205, 12, 19, 236, 95, 151, 68, 23, 196, 167, 126, 61, 100, 93, 25, 115, 96, 129, 79, 220, 34, 42, 144, 136, 70, 238, 184, 20, 222, 94, 11, 219, 224, 50, 58, 10, 73, 6, 36, 92, 194, 211, 172, 98, 145, 149, 228, 121, 231, 200, 55, 109, 141, 213, 78, 169, 108, 86, 244, 234, 101, 122, 174, 8, 186, 120, 37, 46, 28, 166, 180, 198, 232, 221, 116, 31, 75, 189, 139, 138, 112, 62, 181, 102, 72, 3, 246, 14, 97, 53, 87, 185, 134, 193, 29, 158, 225, 248, 152, 17, 105, 217, 142, 148, 155, 30, 135, 233, 206, 85, 40, 223, 140, 161, 137, 13, 191, 230, 66, 104, 65, 153, 45, 15, 176, 84, 187, 22];

invSBox(SBox(1:256) + 1) = 0:255;

Weight_Hamming_vect = [0 1 1 2 1 2 2 3 1 2 2 3 2 3 3 4 1 2 2 3 2 3 3 4 2 3 3 4 3 4 4 5 1 2 2 3 2 3 3 4 2 3 3 4 3 4 4 5 2 3 3 4 3 4 4 5 3 4 4 5 4 5 5 6 1 2 2 3 2 3 3 4 2 3 3 4 3 4 4 5 2 3 3 4 3 4 4 5 3 4 4 5 4 5 5 6 2 3 3 4 3 4 4 5 3 4 4 5 4 5 5 6 3 4 4 5 4 5 5 6 4 5 5 6 5 6 6 7 1 2 2 3 2 3 3 4 2 3 3 4 3 4 4 5 2 3 3 4 3 4 4 5 3 4 4 5 4 5 5 6 2 3 3 4 3 4 4 5 3 4 4 5 4 5 5 6 3 4 4 5 4 5 5 6 4 5 5 6 5 6 6 7 2 3 3 4 3 4 4 5 3 4 4 5 4 5 5 6 3 4 4 5 4 5 5 6 4 5 5 6 5 6 6 7 3 4 4 5 4 5 5 6 4 5 5 6 5 6 6 7 4 5 5 6 5 6 6 7 5 6 6 7 6 7 7 8];
